In [50]:
import re
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

In [51]:
#set parameters
rsrpThr = -110
minDist = 0.02
minPts = 5

In [52]:
oprBands = ['TWM-n78', 'FET-n78','CHT-n78','CHT-n1']

In [53]:
#read excel
scanner = pd.read_excel("c:/Work/Operators/TWM/scanner/TaoYuan.xlsx", sheet_name="Series Formatted Data")

In [54]:
scanner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64649 entries, 0 to 64648
Data columns (total 13 columns):
 #   Column                                              Non-Null Count  Dtype         
---  ------                                              --------------  -----         
 0   Message                                             64649 non-null  int64         
 1   Time                                                64649 non-null  datetime64[ns]
 2   Distance                                            64640 non-null  float64       
 3   Longitude                                           64649 non-null  float64       
 4   Latitude                                            64649 non-null  float64       
 5   NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_636000_0      2161 non-null   float64       
 6   NR_Scan_SS_RSRP_SortedBy_RSRP_for_NRARFCN_636000_0  2161 non-null   float64       
 7   NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_625324_0      24196 non-null  float64       
 8   NR_Sca

In [55]:
scanner.describe()

,Message,Distance,Longitude,Latitude,NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_636000_0,NR_Scan_SS_RSRP_SortedBy_RSRP_for_NRARFCN_636000_0,NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_625324_0,NR_Scan_SS_RSRP_SortedBy_RSRP_for_NRARFCN_625324_0,NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_631000_0,NR_Scan_SS_RSRP_SortedBy_RSRP_for_NRARFCN_631000_0,NR_Scan_PCI_SortedBy_RSRP_for_NRARFCN_431570_0,NR_Scan_SS_RSRP_SortedBy_RSRP_for_NRARFCN_431570_0
count,64649.000000,64640.000000,64649.000000,64649.000000,2161.000000,2161.000000,24196.000000,24196.000000,22560.000000,22560.000000,1225.000000,1225.000000
mean,343560.023202,14882.008782,121.240796,24.971555,259.533549,-121.712573,260.270210,-110.030576,204.841179,-107.708181,319.186122,-95.311167
std,170621.050905,10172.260328,0.075036,0.066304,134.677299,13.255892,137.676193,16.410734,130.445943,17.912436,132.757474,16.056675
min,52.000000,0.000000,121.008890,24.817510,16.000000,-130.500000,1.000000,-130.300000,0.000000,-131.100000,12.000000,-120.730000
25%,208401.000000,6210.192871,121.202690,24.920210,144.000000,-127.900000,139.000000,-123.200000,85.000000,-122.900000,339.000000,-106.830000
50%,373366.000000,13625.370606,121.245960,24.973260,266.000000,-125.670000,265.000000,-115.230000,217.000000,-113.175000,339.000000,-99.800000
75%,479890.000000,21946.610840,121.296460,25.023960,383.000000,-121.700000,356.000000,-100.300000,315.000000,-94.700000,347.000000,-88.300000
max,603352.000000,43433.179688,121.406260,25.111660,463.000000,-43.600000,490.000000,-47.900000,445.000000,-46.450000,848.000000,-49.170000


In [56]:
#change column names: only keep  PCI, RSRP and the corresponding ARFCN
oldColName = scanner.columns
pattern = r"(NR_Scan_|_SortedBy_RSRP_for_NRARFCN_|_0|SS_)"
newColName = {}

#create oldColName to newColName mapping dictionary
for colName in oldColName:
    temp = re.sub(pattern, "", colName)
    newColName[colName] = temp

#rename coluimn names
scanner = scanner.rename(columns = newColName)

In [57]:
scanner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64649 entries, 0 to 64648
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Message     64649 non-null  int64         
 1   Time        64649 non-null  datetime64[ns]
 2   Distance    64640 non-null  float64       
 3   Longitude   64649 non-null  float64       
 4   Latitude    64649 non-null  float64       
 5   PCI636000   2161 non-null   float64       
 6   RSRP636000  2161 non-null   float64       
 7   PCI625324   24196 non-null  float64       
 8   RSRP625324  24196 non-null  float64       
 9   PCI631000   22560 non-null  float64       
 10  RSRP631000  22560 non-null  float64       
 11  PCI431570   1225 non-null   float64       
 12  RSRP431570  1225 non-null   float64       
dtypes: datetime64[ns](1), float64(11), int64(1)
memory usage: 6.4 MB


In [58]:
scanner.head(100)

,Message,Time,Distance,Longitude,Latitude,PCI636000,RSRP636000,PCI625324,RSRP625324,PCI631000,RSRP631000,PCI431570,RSRP431570
0,442813,2020-07-03 13:10:51.075,10637.201172,121.00889,24.94891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,442806,2020-07-03 13:10:49.313,10634.145508,121.00889,24.94894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,442800,2020-07-03 13:10:47.323,10631.259766,121.00889,24.94896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,442796,2020-07-03 13:10:46.195,10628.789063,121.00889,24.94899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,442817,2020-07-03 13:10:52.273,10640.853516,121.00891,24.94889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,442572,2020-07-03 13:09:43.318,9407.902344,121.01359,24.95896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,442570,2020-07-03 13:09:42.373,9380.840820,121.01364,24.95921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,442566,2020-07-03 13:09:41.195,9354.041016,121.01369,24.95944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,442562,2020-07-03 13:09:40.125,9327.282227,121.01374,24.95969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
#wide to long
scannerL = pd.wide_to_long(scanner, ['PCI', 'RSRP'], j = "ARFCN", i=["Longitude", "Latitude"])
scannerL.head()

Distance                    Time  Message  PCI  \
Longitude Latitude ARFCN                                                        
121.00889 24.94891 636000  10637.201172 2020-07-03 13:10:51.075   442813  NaN   
                   625324  10637.201172 2020-07-03 13:10:51.075   442813  NaN   
                   631000  10637.201172 2020-07-03 13:10:51.075   442813  NaN   
                   431570  10637.201172 2020-07-03 13:10:51.075   442813  NaN   
          24.94894 636000  10634.145508 2020-07-03 13:10:49.313   442806  NaN   

                           RSRP  
Longitude Latitude ARFCN         
121.00889 24.94891 636000   NaN  
                   625324   NaN  
                   631000   NaN  
                   431570   NaN  
          24.94894 636000   NaN

In [60]:
#remove NaN
scannerL = scannerL.dropna(axis=0,how="any")
scannerL.describe()

,Distance,Message,PCI,RSRP
count,50142.000000,50142.000000,50142.000000,50142.000000
mean,14074.542163,268309.372602,236.739061,-109.129541
std,8905.242761,168833.145388,137.576192,17.348290
min,0.012355,358.000000,0.000000,-131.100000
25%,6221.939453,129939.500000,122.000000,-123.400000
50%,13646.016602,242396.000000,230.000000,-114.670000
75%,20864.475586,402477.250000,356.000000,-97.930000
max,42497.160807,602700.000000,848.000000,-43.600000


In [61]:
#set lon, lat, ARFCN to column
scannerL = scannerL.reset_index(['Longitude', 'Latitude', 'ARFCN'])
scannerL.head()

,Longitude,Latitude,ARFCN,Distance,Time,Message,PCI,RSRP
0,121.05681,24.99994,631000,22649.906250,2020-07-03 13:23:04.127,445424,401.0,-128.10
1,121.05721,24.99969,631000,22601.187500,2020-07-03 13:23:02.349,445415,401.0,-129.67
2,121.05741,24.99954,631000,22576.142578,2020-07-03 13:23:01.218,445409,401.0,-129.10
3,121.05761,24.99941,631000,22551.097656,2020-07-03 13:23:00.290,445404,401.0,-129.20
4,121.05801,24.99916,631000,22501.207031,2020-07-03 13:22:58.460,445396,401.0,-128.60


In [62]:
scannerL.describe()

,Longitude,Latitude,ARFCN,Distance,Message,PCI,RSRP
count,50142.000000,50142.000000,50142.000000,50142.000000,50142.000000,50142.000000,50142.000000
mean,121.268127,24.975153,623604.338758,14074.542163,268309.372602,236.739061,-109.129541
std,0.050609,0.044765,30555.015635,8905.242761,168833.145388,137.576192,17.348290
min,121.056810,24.818860,431570.000000,0.012355,358.000000,0.000000,-131.100000
25%,121.226660,24.951440,625324.000000,6221.939453,129939.500000,122.000000,-123.400000
50%,121.275250,24.976250,625324.000000,13646.016602,242396.000000,230.000000,-114.670000
75%,121.307940,25.000960,631000.000000,20864.475586,402477.250000,356.000000,-97.930000
max,121.406160,25.110660,636000.000000,42497.160807,602700.000000,848.000000,-43.600000


In [63]:
scannerL['ARFCN'].unique()

array([631000, 431570, 625324, 636000], dtype=int64)

In [64]:
#add OperatorBand column
scannerL['OperatorBand'] = np.select(
    [
        scannerL['ARFCN'] == 636000,
        scannerL['ARFCN'] == 625324,
        scannerL['ARFCN'] == 631000,
        scannerL['ARFCN'] == 431570
    ],
    [
        "TWM-n78",
        "FET-n78",
        "CHT-n78",
        "CHT-n1"
    ]
    
)

In [65]:
scannerL.head()

,Longitude,Latitude,ARFCN,Distance,Time,Message,PCI,RSRP,OperatorBand
0,121.05681,24.99994,631000,22649.906250,2020-07-03 13:23:04.127,445424,401.0,-128.10,CHT-n78
1,121.05721,24.99969,631000,22601.187500,2020-07-03 13:23:02.349,445415,401.0,-129.67,CHT-n78
2,121.05741,24.99954,631000,22576.142578,2020-07-03 13:23:01.218,445409,401.0,-129.10,CHT-n78
3,121.05761,24.99941,631000,22551.097656,2020-07-03 13:23:00.290,445404,401.0,-129.20,CHT-n78
4,121.05801,24.99916,631000,22501.207031,2020-07-03 13:22:58.460,445396,401.0,-128.60,CHT-n78


In [66]:
scannerLfRsrp = scannerL[scannerL['RSRP'] > rsrpThr ]
scannerLfRsrp.describe()

,Longitude,Latitude,ARFCN,Distance,Message,PCI,RSRP
count,20457.000000,20457.000000,20457.000000,20457.000000,20457.000000,20457.000000,20457.000000
mean,121.272246,24.984084,618393.942709,14246.492677,243610.286259,220.120448,-91.500381
std,0.044024,0.036178,43133.664117,8425.539717,156683.710267,138.719862,13.089641
min,121.062810,24.818860,431570.000000,0.217942,2980.000000,0.000000,-109.980000
25%,121.228910,24.958460,625324.000000,7807.859375,121229.000000,90.000000,-102.950000
50%,121.294060,24.987760,625324.000000,13427.944336,229066.000000,220.000000,-93.330000
75%,121.309760,25.001360,631000.000000,20333.113281,311952.000000,344.000000,-81.350000
max,121.406160,25.110660,636000.000000,41313.558594,598156.000000,848.000000,-43.600000


In [67]:
scannerL.groupby('OperatorBand').PCI.nunique()

OperatorBand
CHT-n1      15
CHT-n78    137
FET-n78     81
TWM-n78     44
Name: PCI, dtype: int64

In [68]:
scannerLfRsrp.groupby('OperatorBand').PCI.nunique()

OperatorBand
CHT-n1      15
CHT-n78    123
FET-n78     75
TWM-n78      5
Name: PCI, dtype: int64

In [128]:
tList = list()

#do DBSCAN on operaterBand and PCI level
for oprBand in oprBands:
    #get unique PCI on oprBands level
    for tpci in scannerLfRsrp[scannerLfRsrp['OperatorBand'] == oprBand]['PCI'].unique():
        temp = scannerLfRsrp[(scannerLfRsrp['OperatorBand'] == oprBand) & (scannerLfRsrp['PCI'] == tpci)]
        #DBSCAN
        db = DBSCAN(eps = minDist, min_samples=minPts).fit(temp[['Latitude', 'Longitude']])
        tdf = pd.concat([temp.reset_index(drop=True), pd.DataFrame(db.labels_, columns= ["clusterID"])], axis=1)
        tList.append(tdf)
        
#Merge all the list into one dataframe
tMerge = pd.concat(tList)            

#remove non clustered data points where clusterID = -1
tMergeCluster = tMerge[tMerge['clusterID'] > -1]

In [139]:
#calculate cell count by operatorBand 
cellCount =  tMergeCluster.groupby(['OperatorBand', 'PCI']).clusterID.nunique().reset_index().groupby('OperatorBand').clusterID.sum().reset_index()
cellCount

,OperatorBand,clusterID
0,CHT-n1,14
1,CHT-n78,123
2,FET-n78,76
3,TWM-n78,2


In [181]:
#calculate the estimated lat lon on cell level which means on operatorBand, PCI, clusterID
tLocation = list()
for oprBand in oprBands:
    #get unique PCI on oprBands level
    for tpci in tMergeCluster[tMergeCluster['OperatorBand'] == oprBand]['PCI'].unique():
        tdf = tMergeCluster[(tMergeCluster['OperatorBand'] == oprBand) & (tMergeCluster['PCI'] == tpci)]
        #get the estimated location of the cell location 
        for cluster in range((tdf['clusterID'].max())+1):
            lat = tdf[tdf['clusterID'] == cluster].sort_values(['RSRP'], ascending = False).head(5)['Longitude'].mean()
            lon = tdf[tdf['clusterID'] == cluster].sort_values(['RSRP'], ascending = False).head(5)['Latitude'].mean()
            d = {'OperatorBand': oprBand, 'PCI': tpci, 'clusterID': cluster,'lon': lon, 'lat': lat}
            tLocation.append(pd.DataFrame(d, index=[0]))
            
tLocation = pd.concat(tLocation)

In [182]:
tLocation

,OperatorBand,PCI,clusterID,lon,lat
0,TWM-n78,144.0,0,25.017278,121.402476
0,TWM-n78,366.0,0,25.018212,121.405944
0,FET-n78,137.0,0,25.063116,121.122208
0,FET-n78,481.0,0,24.911626,121.145704
0,FET-n78,6.0,0,24.986268,121.212334
...,...,...,...,...,...
0,CHT-n1,452.0,0,25.081238,121.235416
0,CHT-n1,451.0,0,25.083392,121.238132
0,CHT-n1,339.0,0,25.089692,121.252608
0,CHT-n1,347.0,0,25.086974,121.254312
